In [1]:
import pandas as pd
import numpy as np
import os
import jieba
import gensim
from wordcloud import WordCloud, ImageColorGenerator
import matplotlib.pyplot as plt
import sys
import pprint
from gensim import corpora, models, similarities

In [4]:
%%time
# Load dataset
data_item = pd.read_excel('../data/推荐系统项目_菜品数据.xlsx')

CPU times: user 6.22 s, sys: 61.2 ms, total: 6.29 s
Wall time: 6.31 s


In [5]:
data_item.item_name = data_item.item_name.astype(str)

In [6]:
# Customized stop words
stoplist = [' ',  '~', '！', '!', '～', '@', '@','#', '#', '\\', '｜', 
            '，', ',', '$', '¥', '%', '%', '^', '……', '&', '&', '*', '*', 
            '(', ')', '.', '。', '（', '）', '_', '—', '+', '+', 
            '`', '·', '{', '}', '「', '」', '[]', ']', '【', '】', 
            ':', '：', '"', '“', '”', '<', '>', '《', '》', '?', '？', '/', '〉']

In [7]:
text_corpus = list(data_item.item_name.values)
text_corpus
# Tokenization
texts = [[word for word in jieba.cut(document, cut_all=False) if word not in stoplist] for document in text_corpus]
# Count word frequencies
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1
processed_corpus = [[token for token in text if frequency[token] > 1] for text in texts]
dictionary = corpora.Dictionary(processed_corpus)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/21/vf965q255vsdl911tn822qzr0000gn/T/jieba.cache
Loading model cost 0.700 seconds.
Prefix dict has been built succesfully.


In [8]:
%%time

def score_simhash(query, doc):
    q_simhash = Simhash(query)
    d_simhash = Simhash(doc)
    max_hashbit = max(len(bin(q_simhash.value)), (len(bin(d_simhash.value))))
    distance = q_simhash.distance(d_simhash)
    similar = 1 - distance/max_hashbit
    return similar

def score_jaccard(query, doc):
    intersection = len(list(set(query).intersection(doc)))
    union = (len(query) + len(doc)) - intersection
    return round(float(intersection) / union, 2)


def score_jaccard2(query, doc):
    s1 = set([i for i in '/'.join(jieba.cut(query)).split('/') if i not in stoplist])
    s2 = set([i for i in '/'.join(jieba.cut(doc)).split('/') if i not in stoplist])
    # print(set([i for i in '/'.join(jieba.cut(query)).split('/') if i not in stoplist]))
    # print(set([i for i in '/'.join(jieba.cut(doc)).split('/') if i not in stoplist]))
    
    intersection = len(list(s1.intersection(s2)))
    
    union = (len(query) + len(doc)) - intersection
    return round(float(intersection) / union, 2)

CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 9.3 µs


> 'q' is the query including searching key words. <br>
'd' refers a document existing in *data_item.item_names*. <br>
The result shows the matching dishes with the corresponding ranking scores(similarity).


In [24]:
ranks = []
q = '豆角'
# d = '香辣牛肉饭'
# score_jaccard2(q, d)

ranks = []
for d in data_item.item_name.values:
    ranks.append([d, score_jaccard(q, d)])
pd.DataFrame(
    sorted(ranks, key=lambda x: x[1], reverse=True)[:50],
    columns = ['Dish name', 'Ranking score']).head(20)

# score_jaccard(q, '香辣牛舌饭')
# for d in data_item.item_name.values[:20000]:
#     ranks.append([d, score_simhash(q, d)])
# print(sorted(ranks, key=lambda x: x[1], reverse=True)[:10])

,Dish name,Ranking score
0,羊角豆,0.67
1,羊角豆,0.67
2,羊角豆,0.67
3,干煸豆角,0.50
4,干煸豆角,0.50
5,干煸豆角,0.50
6,干煸豆角,0.50
7,茄子豆角,0.50
8,酸豆角面,0.50
9,酸豆角面,0.50
